In [ ]:
!unzip /content/train.zip

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation

In [ ]:
data_dir = '/content/train'

In [46]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary',
    subset='training',
    color_mode='rgb'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    color_mode='rgb'
)

Found 164 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Dense

inputs = Input(shape=(28, 28, 3))
x = MaxPooling2D(pool_size=(2, 2))(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)  # Используем 2 выхода для 2 классов

model = Model(inputs, x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 3)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 588)               0         
                                                                 
 dense (Dense)               (None, 64)                37696     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 37826 (147.76 KB)
Trainable params: 37826 (147.76 KB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

In [49]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss}, validation accuracy: {val_accuracy}')

2/2 [==============================] - 0s 11ms/step - loss: 0.6758 - accuracy: 0.6250
Validation loss: 0.6758458018302917, validation accuracy: 0.625


In [ ]:
model.save('model_rgb.h5')

In [54]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

test_image_path = '/content/vitalik.jpeg'
img = load_img(test_image_path, target_size=(28, 28))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]

class_names = ['cat','dog']
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 40ms/step
Predicted class: dog


In [ ]:
!pip install -U coremltools


In [52]:
import coremltools as ct



image_input = ct.ImageType(shape=(1, 28, 28, 3))



coreml_model = ct.convert(
    model,
    inputs=[image_input],
    classifier_config=ct.ClassifierConfig(['cat', 'dog']),
    minimum_deployment_target=ct.target.iOS13
)


coreml_model.save('model_cat_dog_rgb.mlmodel')


Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 19/19 [00:00<00:00, 3871.92 ops/s]


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [79]:
image_path = "/content/vitalik_resized_rgb.jpeg"
image = cv2.imread(image_path)

if image is None:
    print("Не удалось загрузить изображение")
else:
    cv2_imshow(image)

    height, width, channels = image.shape
    print("Ширина изображения:", width)
    print("Высота изображения:", height)
    print("Количество каналов изображения:", channels)


Ширина изображения: 28
Высота изображения: 28
Количество каналов изображения: 3


#Making input.json

In [56]:
from PIL import Image
import numpy as np
import json

In [57]:
img_path = '/content/vitalik_resized_rgb.jpeg'
img = Image.open(img_path)

In [61]:
from PIL import Image
import numpy as np
import json




In [62]:
# Load the image
img_path = '/content/vitalik_resized_rgb.jpeg'
img = Image.open(img_path)


In [65]:
img_matrix = np.array(img)


In [ ]:
img_matrix

In [68]:
sample_input = img_matrix / 255.0

In [69]:
X = [[[int(x * float(10**18))] for x in row] for row in sample_input]



In [73]:
np.shape(X)

(28, 28, 1)

In [70]:
# Define the path for the output JSON file
output_json_path = '/content/input.json'

# Save the matrix in the required format as JSON with the key "in"
with open(output_json_path, 'w') as f:
    json.dump({"in": X}, f)


In [80]:
from PIL import Image
import numpy as np
import json

# Define the path of the image
img_path = '/content/vitalik_resized_rgb.jpeg'

# Load the image
img = cv2.imread(image_path)

# Convert image to a numpy array
img_array = np.array(img)



In [81]:
sample_input = img_array / 255.0

In [82]:
np.shape(sample_input)

(28, 28, 3)

In [83]:


# Convert the normalized pixel values to the required format
# Ensuring each channel value of the pixel is wrapped in a list
X = [[[int(channel_value * float(10**18)) for channel_value in pixel] for pixel in row] for row in sample_input]


In [86]:

# Define the path for the output JSON file
output_json_path = '/content/input.json'

# Save the matrix in the required format as JSON with the key "in"
with open(output_json_path, 'w') as f:
    json.dump({"in": X}, f)

output_json_path


'/content/input.json'